In [2]:
import backtrader as bt
import pandas as pd
from datetime import datetime
import numpy as np

In [3]:
def get_all_date_data(start_time, end_time, list_assets):
    data_path = 'etf_sina'

    # 从本地保存的数据中读出需要的股票日数据
    list_all = []
    for c in list_assets:
        df = pd.read_csv(f'{data_path}/{c}.csv')
        df['asset'] = c
        list_all.append(df[(df['date'] >= start_time) & (df['date'] <= end_time)])
        
    print(len(list_all))

    # 所有股票日数据拼接成一张表
    df_all = pd.concat(list_all) 

    # 返回计算因子需要的列
    df_all = df_all.reset_index()
    df_all = df_all[['date', 'asset', "open", "close", "high", "low", "volume", "pctChg"]]
    return df_all

In [4]:
fund_etf_top300_sina = pd.read_csv('fund_etf_top300_sina.csv')

In [5]:
list_assets = fund_etf_top300_sina['代码'].to_list()

In [6]:
list_assets = list_assets[:30]

In [7]:
start_time = '2023-11-01'
end_time = '2024-12-31'
fromdate = datetime.strptime(start_time, '%Y-%m-%d')
todate = datetime.strptime(end_time, '%Y-%m-%d')

In [8]:
df_all = get_all_date_data(start_time, end_time, list_assets)

30


In [9]:
df1 = df_all.rename(columns={
        "date": "datetime", 
        "asset": "sec_code"})
df1["openinterest"] = 0
daily_price=df1[['sec_code','datetime', "open", "close", "high", "low", "volume", 'openinterest']]
daily_price['datetime'] = pd.to_datetime(daily_price['datetime'])
daily_price = daily_price.set_index(['datetime'])
daily_price

/tmp/ipykernel_21203/2658912843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_price['datetime'] = pd.to_datetime(daily_price['datetime'])


,sec_code,open,close,high,low,volume,openinterest
datetime,,,,,,,
2023-11-01,sh511990,99.996,100.005,100.007,99.994,134380654,0
2023-11-02,sh511990,99.997,100.004,100.006,99.997,112003561,0
2023-11-03,sh511990,100.008,100.001,100.010,99.996,197399026,0
2023-11-06,sh511990,99.993,99.989,99.995,99.983,168977782,0
2023-11-07,sh511990,99.989,99.998,99.999,99.985,111036776,0
...,...,...,...,...,...,...,...
2024-12-25,sh511010,141.254,140.995,141.291,140.953,2063778,0
2024-12-26,sh511010,141.025,140.997,141.087,140.987,2875300,0
2024-12-27,sh511010,141.120,141.288,141.299,141.066,2903900,0


In [11]:
stock = 'sh512050'
data = pd.DataFrame(index=daily_price.index.unique())
df = daily_price.query(f"sec_code=='{stock}'")[['open','high','low','close','volume','openinterest']]
data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].bfill()

In [12]:
data_

,open,high,low,close,volume,openinterest
datetime,,,,,,
2023-11-01,0.982,0.988,0.964,0.966,0.000000e+00,0.0
2023-11-02,0.982,0.988,0.964,0.966,0.000000e+00,0.0
2023-11-03,0.982,0.988,0.964,0.966,0.000000e+00,0.0
2023-11-06,0.982,0.988,0.964,0.966,0.000000e+00,0.0
2023-11-07,0.982,0.988,0.964,0.966,0.000000e+00,0.0
...,...,...,...,...,...,...
2024-12-25,0.963,0.967,0.958,0.963,3.529156e+09,0.0
2024-12-26,0.963,0.966,0.959,0.965,3.411310e+09,0.0
2024-12-27,0.964,0.972,0.961,0.963,4.346913e+09,0.0


In [247]:


# class VwapRollingIndicator(bt.Indicator):
#     """
#     Volume Weighted Average Price (VWAP) indicator, rolling calculation.
#     """

#     lines = ("vwap_rolling",)
#     params = {"period": 14}
#     plotinfo = {"subplot": False}
#     plotlines = {"vwap_rolling": {"color": "green"}}

#     def __init__(self) -> None:
#         self.hlc = (self.data.high + self.data.low + self.data.close) / 3.0
#         self.hlc_volume_sum = bt.ind.SumN(self.hlc * self.data.volume, period=self.p.period)
#         self.volume_sum = bt.ind.SumN(self.data.volume, period=self.p.period)

#         self.lines.vwap_rolling = bt.DivByZero(self.hlc_volume_sum, self.volume_sum, None) 


In [13]:
daily_price

,sec_code,open,close,high,low,volume,openinterest
datetime,,,,,,,
2023-11-01,sh511990,99.996,100.005,100.007,99.994,134380654,0
2023-11-02,sh511990,99.997,100.004,100.006,99.997,112003561,0
2023-11-03,sh511990,100.008,100.001,100.010,99.996,197399026,0
2023-11-06,sh511990,99.993,99.989,99.995,99.983,168977782,0
2023-11-07,sh511990,99.989,99.998,99.999,99.985,111036776,0
...,...,...,...,...,...,...,...
2024-12-25,sh511010,141.254,140.995,141.291,140.953,2063778,0
2024-12-26,sh511010,141.025,140.997,141.087,140.987,2875300,0
2024-12-27,sh511010,141.120,141.288,141.299,141.066,2903900,0


In [14]:

cerebro = bt.Cerebro(stdstats=False)

for stock in daily_price['sec_code'].unique():
    # 日期对齐
    data = pd.DataFrame(index=daily_price.index.unique())
    df = daily_price.query(f"sec_code=='{stock}'")[['open','high','low','close','volume','openinterest']]
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].bfill()
    datafeed = bt.feeds.PandasData(dataname=data_, fromdate=fromdate, todate=todate)
    cerebro.adddata(datafeed, name=stock)


class MultiStrategy(bt.Strategy):
    params = (
        ('sma_fast', 10),
        ('sma_slow', 30),
        ('rsi_period', 14),
        ('rsi_overbought', 70),
        ('rsi_oversold', 30),
        ('macd_fast', 12),
        ('macd_slow', 26),
        ('macd_signal', 9),
        ('bollinger_period', 20),
        ('devfactor', 2),
        ('atr_period', 14),
        ('adx_period', 14),
        ('stoploss_atr_mult', 2),      # ATR倍数用于止损
        ('takeprofit_atr_mult', 4),    # ATR倍数用于止盈
        ('risk_per_trade', 0.02),      # 每笔交易风险比例
        ('vwap_period', 20),
        ('obv_ema', 21),
        ('ichimoku_tenkan', 9),
        ('ichimoku_kijun', 26),
        ('supertrend_period', 10),
        ('supertrend_multiplier', 3),
        ('min_atr', 0.2),
        ('trailing_stop', True),
        ('trail_atr_mult', 1.5),       # 改为基于ATR的移动止损乘数
        ('max_positions', 5),
        ('volume_filter', True),
        ('volume_threshold', 1.5),
        ('use_indicators_cache', True)  # 启用指标缓存以提高性能
    )

    def __init__(self):
        self.orders = {}        # 订单管理
        self.indicators = {}    # 指标存储
        self.stops = {}         # 止损点管理
        self.position_count = 0 # 当前持仓数量
        self.performance = {}   # 表现记录
        self._indicators_cache = {} # 指标计算缓存
        
        # 初始化每个数据源的指标
        for d in self.datas:
            # 基础指标字典
            ind = self.indicators[d] = {}
            
            # --- 关键指标计算 ---
            # 波动率指标 - ATR是仓位管理的核心
            ind['atr'] = bt.indicators.ATR(d, period=self.p.atr_period)
            
            # --- 移动平均线指标 ---
            ind['sma_fast'] = bt.indicators.SMA(d.close, period=self.p.sma_fast)
            ind['sma_slow'] = bt.indicators.SMA(d.close, period=self.p.sma_slow)
            ind['ema_fast'] = bt.indicators.EMA(d.close, period=self.p.sma_fast)
            ind['ema_slow'] = bt.indicators.EMA(d.close, period=self.p.sma_slow)
            
            # --- 震荡类指标 ---
            ind['rsi'] = bt.indicators.RelativeStrengthIndex(d.close, period=self.p.rsi_period, safediv=True)
            ind['stochastic'] = bt.indicators.Stochastic(d, period=14, period_dfast=3, period_dslow=3)
            
            # --- 趋势类指标 ---
            ind['macd'] = bt.indicators.MACD(
                d, period_me1=self.p.macd_fast,
                period_me2=self.p.macd_slow,
                period_signal=self.p.macd_signal
            )
            ind['adx'] = bt.indicators.AverageDirectionalMovementIndex(d, period=self.p.adx_period)
            
            # --- 波动率指标 ---
            ind['bollinger'] = bt.indicators.BollingerBands(
                d, period=self.p.bollinger_period, devfactor=self.p.devfactor
            )
            
            # --- 成交量指标 ---
            ind['volume_sma'] = bt.indicators.SMA(d.volume, period=10)
            ind['volume_ma20'] = bt.indicators.SMA(d.volume, period=20)
            ind['obv'] = bt.talib.OBV([d.close, d.volume])
            ind['obv_ema'] = bt.indicators.EMA(ind['obv'], period=self.p.obv_ema)
            
            # --- 价格行为指标 ---
            ind['heikin_ashi'] = bt.indicators.HeikinAshi(d)
            
            # --- 复合指标 ---
            # 添加VWAP
            ind['vwap'] = VwapRollingIndicator(d, period=self.p.vwap_period)
            
            # 一目均衡表
            ind['ichimoku'] = bt.indicators.Ichimoku(
                d, 
                tenkan=self.p.ichimoku_tenkan,
                kijun=self.p.ichimoku_kijun,
                senkou=52,
                chikou=26
            )
            
            # SuperTrend指标计算 - 优化为使用已计算的ATR
            atr = ind['atr']
            multiplier = self.p.supertrend_multiplier
            
            # 计算上轨和下轨
            upperband = ((d.high + d.low) / 2) + (multiplier * atr)
            lowerband = ((d.high + d.low) / 2) - (multiplier * atr)
            
            ind['supertrend_upper'] = upperband
            ind['supertrend_lower'] = lowerband
            
            # --- 信号合成 ---
            # 使用指标值生成延迟信号 - 提高性能
            if self.p.use_indicators_cache:
                # 信号将在next()中根据实际指标值计算
                pass
            else:
                # 使用传统的信号合成方式
                self._calculate_signals(d, ind)
            
            # 初始化股票表现记录
            self.performance[d._name] = {
                'wins': 0,
                'losses': 0,
                'total_profit': 0,
                'total_loss': 0
            }

    def _calculate_signals(self, data, ind):
        """计算交易信号 - 分离出来以便优化性能"""
        # 趋势上升信号
        ind['trend_up'] = bt.And(
            (ind['sma_fast'] > ind['sma_slow']),
            (ind['macd'].macd > ind['macd'].signal),
            (data.close > ind['bollinger'].mid)
        )
        
        ind['trend_strong'] = bt.And(
            ind['trend_up'],
            (ind['adx'] > 25),
            # (data.close > ind['vwap'])
        )

        ind['reversal_down'] = bt.And(
            (ind['rsi'] > self.p.rsi_overbought),
            (data.close > ind['bollinger'].top),
            (ind['obv'] < ind['obv_ema'])
        )
        
        ind['reversal_up'] = bt.And(
            (ind['rsi'] < self.p.rsi_oversold),
            (data.close < ind['bollinger'].bot),
            (ind['obv'] > ind['obv_ema'])
        )

        ind['momentum_signal'] = bt.And(
            (ind['adx'] > 25),
            (ind['volume_sma'] > ind['volume_ma20'] * self.p.volume_threshold if self.p.volume_filter else True)
        )
        
        # 一目均衡表多头信号
        ind['ichimoku_bullish'] = bt.And(
            (data.close > ind['ichimoku'].senkou_span_a),
            (data.close > ind['ichimoku'].senkou_span_b),
            (ind['ichimoku'].tenkan_sen > ind['ichimoku'].kijun_sen)
        )
        
        # 自定义SuperTrend多头信号
        ind['supertrend_bullish'] = data.close > ind['supertrend_lower']
        
        # 组合信号 - 多头进场
        ind['long_entry'] = bt.Or(
            bt.And(ind['trend_strong'], ind['momentum_signal']),
            bt.And(ind['reversal_up'], ind['momentum_signal']),
            bt.And(ind['ichimoku_bullish'], ind['supertrend_bullish'])
        )
        
        # 组合信号 - 多头出场
        ind['long_exit'] = bt.Or(
            ind['reversal_down'],
            (ind['macd'].macd < ind['macd'].signal),
            (data.close < ind['supertrend_lower'])
        )

    def check_signal(self, data, signal_type):
        """基于指标原始值实时计算信号 - 提高性能"""
        ind = self.indicators[data]
        
        # 缓存键
        cache_key = (data._name, signal_type, data.datetime.datetime(0).isoformat())
        
        # 如果已经计算过，直接返回缓存结果
        if self.p.use_indicators_cache and cache_key in self._indicators_cache:
            return self._indicators_cache[cache_key]
            
        result = False
        
        if signal_type == 'trend_up':
            result = (ind['sma_fast'][0] > ind['sma_slow'][0] and 
                     ind['macd'].macd[0] > ind['macd'].signal[0] and 
                     data.close[0] > ind['bollinger'].mid[0])
                     
        elif signal_type == 'trend_strong':
            result = (self.check_signal(data, 'trend_up') and 
                     ind['adx'][0] > 25 
                    #  and data.close[0] > ind['vwap'][0]
                     )
                     
        elif signal_type == 'reversal_down':
            result = (ind['rsi'][0] > self.p.rsi_overbought and 
                     data.close[0] > ind['bollinger'].top[0] and 
                     ind['obv'][0] < ind['obv_ema'][0])
                     
        elif signal_type == 'reversal_up':
            result = (ind['rsi'][0] < self.p.rsi_oversold and 
                     data.close[0] < ind['bollinger'].bot[0] and 
                     ind['obv'][0] > ind['obv_ema'][0])
                     
        elif signal_type == 'momentum_signal':
            volume_cond = True
            if self.p.volume_filter:
                volume_cond = ind['volume_sma'][0] > ind['volume_ma20'][0] * self.p.volume_threshold
            result = ind['adx'][0] > 25 and volume_cond
            
        elif signal_type == 'ichimoku_bullish':
            result = (data.close[0] > ind['ichimoku'].senkou_span_a[0] and 
                     data.close[0] > ind['ichimoku'].senkou_span_b[0] and 
                     ind['ichimoku'].tenkan_sen[0] > ind['ichimoku'].kijun_sen[0])
                     
        elif signal_type == 'supertrend_bullish':
            result = data.close[0] > ind['supertrend_lower'][0]
            
        elif signal_type == 'long_entry':
            result = (
                (self.check_signal(data, 'trend_strong') and self.check_signal(data, 'momentum_signal')) or
                (self.check_signal(data, 'reversal_up') and self.check_signal(data, 'momentum_signal')) or
                (self.check_signal(data, 'ichimoku_bullish') and self.check_signal(data, 'supertrend_bullish'))
            )
            
        elif signal_type == 'long_exit':
            result = (
                self.check_signal(data, 'reversal_down') or
                ind['macd'].macd[0] < ind['macd'].signal[0] or
                data.close[0] < ind['supertrend_lower'][0]
            )
            
        # 存储计算结果到缓存
        if self.p.use_indicators_cache:
            self._indicators_cache[cache_key] = result
            
        return result

    def next(self):
        """每个bar执行的主要策略逻辑"""
        # 更新当前持仓数量
        self.position_count = sum(1 for d in self.datas if self.getposition(d).size > 0)
        
        # 清理过期缓存以避免内存泄漏
        if self.p.use_indicators_cache and len(self._indicators_cache) > 1000:
            self._indicators_cache.clear()
        
        for d in self.datas:
            pos = self.getposition(d).size
            ind = self.indicators[d]

            # 确保有足够的历史数据用于计算指标
            min_period = max(
                self.p.atr_period, 
                self.p.sma_slow, 
                self.p.bollinger_period,
                self.p.ichimoku_kijun + 26  # 一目均衡表需要更长的历史数据
            )

            # 数据有效性检查
            if len(d) < min_period or np.isnan(d.close[0]) or np.isnan(ind['atr'][0]) or ind['atr'][0] <= 0:
                continue

            # 更新移动止损
            if pos > 0 and self.p.trailing_stop:
                self.update_trailing_stop(d, ind)

            # ---- 仓位管理逻辑 ----
            if pos == 0:  # 无仓位时，考虑开仓
                # 如果已经达到最大持仓数量，不开新仓
                if self.position_count >= self.p.max_positions:
                    continue
                    
                # 多头进场信号 - 使用优化后的信号计算
                if self.check_signal(d, 'long_entry'):
                    # 计算仓位大小 - 只使用ATR
                    size = self.calculate_position_size_atr(d, ind)
                    if size > 0:
                        self.open_long(d, size, ind)
            else:  # 有仓位时，考虑平仓
                if self.check_signal(d, 'long_exit'):
                    self.close_position(d)

    def calculate_position_size_atr(self, data, ind):
        """基于ATR的仓位计算 - 优化版本"""
        # 获取当前价格
        price = data.close[0]
        
        # 确保ATR值有效
        atr_value = max(ind['atr'][0], self.p.min_atr)
        
        # 账户风险金额 = 账户总价值 * 每笔交易风险比例
        risk_amount = self.broker.getvalue() * self.p.risk_per_trade
        
        # 计算止损点 (价格 - ATR乘数 * ATR)
        stop_loss = price - (self.p.stoploss_atr_mult * atr_value)
        
        # 每股风险 = 当前价格 - 止损价格
        risk_per_share = price - stop_loss
        
        # 确保风险合理
        if risk_per_share <= 0 or risk_per_share > price * 0.1:  # 防止止损过大或过小
            risk_per_share = price * 0.02  # 默认2%止损
        
        # 计算仓位大小 = 风险金额 / 每股风险
        size = int(risk_amount / risk_per_share) if risk_per_share > 0 else 0
        
        # 确保size至少为1且不超过可用资金
        cash = self.broker.get_cash()
        return max(1, min(size, int(cash / price) if price > 0 else 0))

    def open_long(self, data, size, ind):
        """开多仓并设置止损止盈 - 基于ATR"""
        price = data.close[0]
        atr_value = max(ind['atr'][0], self.p.min_atr)
        
        # 基于ATR的止损和止盈
        sl = price - (self.p.stoploss_atr_mult * atr_value)
        tp = price + (self.p.takeprofit_atr_mult * atr_value)
        
        # 确保止损合理
        if sl >= price or np.isnan(sl):
            sl = price * 0.98  # 默认2%止损
            
        # 记录止损价格用于跟踪止损
        self.stops[data._name] = sl
        
        # 主订单（开多仓）
        main_order = self.buy(data=data, size=size)
        
        # 止损单
        stop_order = self.sell(data=data, exectype=bt.Order.Stop, 
                 price=sl, size=size, parent=main_order)
                 
        # 止盈单
        take_order = self.sell(data=data, exectype=bt.Order.Limit, 
                 price=tp, size=size, parent=main_order)
                 
        # 存储订单引用以便后续管理
        main_order.stop_order = stop_order
        main_order.take_order = take_order
        
        self.log(f'开仓 {data._name}, 价格: {price:.2f}, 数量: {size}, '
                f'止损: {sl:.2f} (ATR×{self.p.stoploss_atr_mult}), 止盈: {tp:.2f} (ATR×{self.p.takeprofit_atr_mult})')

    def update_trailing_stop(self, data, ind):
        """基于ATR的移动止损"""
        if data._name not in self.stops:
            return
            
        price = data.close[0]
        current_stop = self.stops[data._name]
        atr_value = ind['atr'][0]
        
        # 计算新的移动止损水平 (当前价格 - ATR的trail_atr_mult倍)
        new_stop = price - (atr_value * self.p.trail_atr_mult)
        
        # 仅在新止损高于当前止损时更新
        if new_stop > current_stop:
            self.stops[data._name] = new_stop
            
            # 取消旧的止损订单
            for order in self.broker.get_orders_open(data):
                if order.exectype == bt.Order.Stop and order.issell():
                    self.cancel(order)
                    
            # 创建新的止损订单
            self.sell(data=data, exectype=bt.Order.Stop, 
                     price=new_stop, size=self.getposition(data).size)
                     
            self.log(f'更新移动止损 {data._name}, 新止损: {new_stop:.2f} (ATR×{self.p.trail_atr_mult})')

    def close_position(self, data):
        """平仓并清理相关订单"""
        # 取消所有未完成订单
        for order in self.broker.get_orders_open(data):
            self.cancel(order)
            
        # 平仓
        self.close(data=data)
        
        # 清理止损记录
        if data._name in self.stops:
            del self.stops[data._name]
            
        self.log(f'平仓 {data._name}, 价格: {data.close[0]:.2f}')

    def log(self, txt, dt=None):
        """记录策略信息"""
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_trade(self, trade):
        """交易结束通知，用于记录业绩统计"""
        if trade.isclosed:
            data_name = trade.data._name
            
            self.log(f'交易结束 {data_name}, 毛利润: {trade.pnl:.2f}, 净利润: {trade.pnlcomm:.2f}')
            
            # 更新表现记录
            if trade.pnlcomm > 0:
                self.performance[data_name]['wins'] += 1
                self.performance[data_name]['total_profit'] += trade.pnlcomm
            else:
                self.performance[data_name]['losses'] += 1
                self.performance[data_name]['total_loss'] += trade.pnlcomm

    def notify_order(self, order):
        """订单状态变更通知"""
        if order.status in [order.Submitted, order.Accepted]:
            return
            
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    f'{order.data._name} 买入执行, 价格: {order.executed.price:.2f}, '
                    f'成本: {order.executed.value:.2f}, 手续费: {order.executed.comm:.2f}'
                )
            elif order.issell():
                self.log(
                    f'{order.data._name} 卖出执行, 价格: {order.executed.price:.2f}, '
                    f'成本: {order.executed.value:.2f}, 手续费: {order.executed.comm:.2f}'
                )
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f'订单被取消/拒绝: {order.data._name}, 状态: {order.getstatusname()}')
            
        # 管理相关联的止损止盈订单
        if hasattr(order, 'take_order') and order.take_order is not None:
            if order.take_order.status != order.Canceled: 
                self.cancel(order.take_order)
                
        elif hasattr(order, 'stop_order') and order.stop_order is not None:
            if order.stop_order.status != order.Canceled:
                self.cancel(order.stop_order)
                
    def stop(self):
        """策略结束时执行，用于打印总结报告"""
        # 计算总体胜率
        total_wins = sum(p['wins'] for p in self.performance.values())
        total_trades = sum(p['wins'] + p['losses'] for p in self.performance.values())
        win_rate = total_wins / total_trades if total_trades > 0 else 0
        
        # 计算总收益
        total_profit = sum(p['total_profit'] for p in self.performance.values())
        total_loss = sum(p['total_loss'] for p in self.performance.values())
        net_profit = total_profit + total_loss  # total_loss已经是负数
        
        # 计算盈亏比
        avg_win = total_profit / total_wins if total_wins > 0 else 0
        avg_loss = abs(total_loss) / (total_trades - total_wins) if (total_trades - total_wins) > 0 else 0
        profit_factor = avg_win / avg_loss if avg_loss > 0 else float('inf')
        
        # 打印报告
        self.log('===== 策略回测报告 =====')
        self.log(f'总交易次数: {total_trades}')
        self.log(f'胜率: {win_rate:.2%}')
        self.log(f'净利润: {net_profit:.2f}')
        self.log(f'盈亏比: {profit_factor:.2f}')
        self.log(f'平均盈利交易: {avg_win:.2f}')
        self.log(f'平均亏损交易: {avg_loss:.2f}')
        
        # 打印每个股票的表现
        self.log('===== 个股表现 =====')
        for name, perf in self.performance.items():
            trades = perf['wins'] + perf['losses']
            if trades > 0:
                win_rate = perf['wins'] / trades
                self.log(f'{name}: 交易次数={trades}, 胜率={win_rate:.2%}, 净利润={perf["total_profit"] + perf["total_loss"]:.2f}')


cerebro.addstrategy(MultiStrategy)
cerebro.broker.setcash(1000000)
cerebro.broker.setcommission(commission=0.001)
# cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
# cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
# cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
# cerebro.addsizer(bt.sizers.PercentSizer, percents=90)

results = cerebro.run()
print(f'最终组合价值: {cerebro.broker.getvalue():.2f}')

NameError: name 'VwapRollingIndicator' is not defined

In [16]:
import backtrader as bt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta


class MultiStockStrategy(bt.Strategy):
    """
    多股票回测策略，整合RSI、MACD、移动平均线等多种指标
    实现智能仓位管理和性能优化
    """

    params = (
        ('rsi_period', 14),          # RSI周期
        ('rsi_overbought', 70),      # RSI超买阈值
        ('rsi_oversold', 30),        # RSI超卖阈值
        ('macd_fast', 12),           # MACD快线周期
        ('macd_slow', 26),           # MACD慢线周期
        ('macd_signal', 9),          # MACD信号线周期
        ('sma_fast', 10),            # 快速移动平均线周期
        ('sma_slow', 30),            # 慢速移动平均线周期
        ('atr_period', 14),          # ATR周期
        ('atr_mult', 2),             # ATR乘数，用于止损
        ('risk_pct', 0.02),          # 每笔交易风险比例
        ('max_pos_size', 0.2),       # 单个股票最大仓位比例
        ('max_stocks', 5),           # 最大持股数量
        ('vwap_period', 14),           # WVAP
    )

    def __init__(self):
        """
        初始化函数，在回测开始前执行一次
        预先计算各种指标，以提高性能
        """
        self.inds = {}
        self.position_size = {}
        self.stops = {}

        # 遍历所有数据馈送（多只股票）
        for i, d in enumerate(self.datas):
            # 存储每个股票的指标到字典中
            self.inds[d] = {
                # 价格和成交量指标
                'close': d.close,
                'volume': d.volume,

                # RSI指标
                'rsi': bt.indicators.RSI(
                    d.close,
                    period=self.params.rsi_period
                ),

                # MACD指标
                'macd': bt.indicators.MACD(
                    d.close,
                    period_me1=self.params.macd_fast,
                    period_me2=self.params.macd_slow,
                    period_signal=self.params.macd_signal
                ),

                # 简单移动平均线
                'sma_fast': bt.indicators.SMA(
                    d.close,
                    period=self.params.sma_fast
                ),
                'sma_slow': bt.indicators.SMA(
                    d.close,
                    period=self.params.sma_slow
                ),

                # 布林带
                'bbands': bt.indicators.BollingerBands(
                    d.close,
                    period=20
                ),

                # 平均真实波幅(ATR)
                'atr': bt.indicators.ATR(
                    d,
                    period=self.params.atr_period
                ),

                # 成交量加权平均价格
                # 'vwap': VwapRollingIndicator(d, period=self.p.vwap_period),

                # 资金流向指标
                'mfi': bt.talib.MFI(
                    [d.high, d.low, d.close, d.volume],
                    period=14
                ),

                # CCI指标
                'cci': bt.indicators.CCI(
                    d,
                    period=20
                ),

                # 抛物线转向指标
                'sar': bt.talib.SAR([d.high, d.low]),

                # 动量指标
                'mom': bt.indicators.MomentumOscillator(
                    d.close,
                    period=10
                ),
            }

            # 添加交易信号
            self.inds[d]['golden_cross'] = bt.indicators.CrossOver(
                self.inds[d]['sma_fast'],
                self.inds[d]['sma_slow']
            )

            self.inds[d]['macd_cross'] = bt.indicators.CrossOver(
                self.inds[d]['macd'].macd,
                self.inds[d]['macd'].signal
            )

            # 初始化仓位大小跟踪
            self.position_size[d] = 0.0
            self.stops[d] = None

        # 股票评分和排名用于选择最佳交易机会
        self.stock_scores = {}
        self.current_positions = 0

    def calc_position_size(self, data):
        """计算基于风险的仓位大小"""
        if not self.position_size[data]:
            # 使用ATR确定止损距离
            atr_val = self.inds[data]['atr'][0]
            stop_price = data.close[0] - self.params.atr_mult * atr_val

            # 基于账户价值和风险百分比计算仓位
            risk_per_share = data.close[0] - stop_price
            if risk_per_share <= 0:
                return 0

            cash = self.broker.get_cash()
            risk_amount = cash * self.params.risk_pct
            shares = int(risk_amount / risk_per_share)

            # 检查是否超过最大仓位限制
            max_shares = int((cash * self.params.max_pos_size) / data.close[0])
            shares = min(shares, max_shares)

            # 更新止损价格
            self.stops[data] = stop_price

            return shares
        return 0

    def score_stock(self, data):
        """对股票的交易潜力进行评分（0-100分）"""
        score = 0
        ind = self.inds[data]

        # RSI评分（超卖区域得分高）
        rsi_val = ind['rsi'][0]
        if rsi_val < self.params.rsi_oversold:
            score += 20
        elif rsi_val > self.params.rsi_overbought:
            score -= 20

        # MACD评分
        if ind['macd_cross'] > 0:  # 金叉
            score += 15
        elif ind['macd_cross'] < 0:  # 死叉
            score -= 15

        # 移动平均线评分
        if ind['golden_cross'] > 0:  # 金叉
            score += 15
        elif ind['golden_cross'] < 0:  # 死叉
            score -= 15

        # 布林带评分
        if data.close[0] < ind['bbands'].lines.bot[0]:
            score += 10  # 价格低于下轨
        elif data.close[0] > ind['bbands'].lines.top[0]:
            score -= 10  # 价格高于上轨

        # CCI评分
        if ind['cci'][0] < -100:
            score += 10  # 超卖
        elif ind['cci'][0] > 100:
            score -= 10  # 超买

        # 动量评分
        if ind['mom'][0] > 0 and ind['mom'][-1] < ind['mom'][0]:
            score += 10  # 动量增加

        # 资金流向评分
        if ind['mfi'][0] < 20:
            score += 10  # 资金流出但可能反转
        elif ind['mfi'][0] > 80:
            score -= 10  # 资金流入但可能反转

        # 成交量评分
        vol_avg = sum(ind['volume'].get(size=5)) / 5
        if ind['volume'][0] > vol_avg * 1.5:
            score += 10  # 量能明显放大

        # 归一化分数到0-100
        return max(0, min(100, score + 50))

    def next(self):
        """
        策略主逻辑，在每个交易日执行一次
        """
        # 计算所有股票的分数
        self.stock_scores = {}
        for data in self.datas:
            # 跳过没有足够历史数据的股票
            if not self.inds[data]['sma_slow'][0]:
                continue

            # 计算评分
            self.stock_scores[data] = self.score_stock(data)

        # 检查持仓数量
        self.current_positions = sum(
            1 for data in self.datas if self.getposition(data).size > 0)

        # 检查止损
        for data in self.datas:
            pos = self.getposition(data)
            if pos.size > 0 and self.stops[data] is not None:
                if data.close[0] < self.stops[data]:
                    self.close(data)
                    self.stops[data] = None
                    self.position_size[data] = 0

        # 按评分排序股票
        sorted_stocks = sorted(
            self.stock_scores.items(),
            key=lambda x: x[1],
            reverse=True
        )

        # 卖出信号检测
        for data in self.datas:
            if not self.getposition(data).size:
                continue

            # 收集卖出条件
            sell_signals = 0
            ind = self.inds[data]

            # RSI超买
            if ind['rsi'][0] > self.params.rsi_overbought:
                sell_signals += 1

            # MACD死叉
            if ind['macd_cross'] < 0:
                sell_signals += 1

            # 均线死叉
            if ind['golden_cross'] < 0:
                sell_signals += 1

            # 价格突破布林带上轨
            if data.close[0] > ind['bbands'].lines.top[0]:
                sell_signals += 1

            # 如果至少两个卖出信号触发，则卖出
            if sell_signals >= 2:
                self.close(data)
                self.stops[data] = None
                self.position_size[data] = 0

        # 买入逻辑
        if self.current_positions < self.params.max_stocks:
            # 遍历评分最高的股票
            for data, score in sorted_stocks:
                # 如果已经持有该股票或评分不够高，则跳过
                if self.getposition(data).size > 0 or score < 70:
                    continue

                # 收集买入条件
                buy_signals = 0
                ind = self.inds[data]

                # RSI超卖反弹
                if ind['rsi'][0] < self.params.rsi_oversold and ind['rsi'][0] > ind['rsi'][-1]:
                    buy_signals += 1

                # MACD金叉
                if ind['macd_cross'] > 0:
                    buy_signals += 1

                # 均线金叉
                if ind['golden_cross'] > 0:
                    buy_signals += 1

                # 价格突破布林带下轨后反弹
                if data.close[-1] < ind['bbands'].lines.bot[-1] and data.close[0] > data.close[-1]:
                    buy_signals += 1

                # 成交量放大
                vol_avg = sum(ind['volume'].get(size=5)) / 5
                if ind['volume'][0] > vol_avg * 1.5:
                    buy_signals += 1

                # 如果至少三个买入信号触发，则买入
                if buy_signals >= 3:
                    size = self.calc_position_size(data)
                    if size > 0:
                        self.buy(data, size=size)
                        self.position_size[data] = size
                        self.current_positions += 1

                # 如果已达到最大持股数量，则停止买入
                if self.current_positions >= self.params.max_stocks:
                    break

    def notify_order(self, order):
        """处理订单状态通知"""
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    f'买入执行: {order.data._name}, 价格: {order.executed.price:.2f}, 成本: {order.executed.value:.2f}, 佣金: {order.executed.comm:.2f}')
            else:
                self.log(
                    f'卖出执行: {order.data._name}, 价格: {order.executed.price:.2f}, 价值: {order.executed.value:.2f}, 佣金: {order.executed.comm:.2f}')

    def notify_trade(self, trade):
        """处理交易结果通知"""
        if trade.isclosed:
            self.log(f'交易利润: 毛利 {trade.pnl:.2f}, 净利 {trade.pnlcomm:.2f}')

    def log(self, txt, dt=None):
        """日志记录函数"""
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def stop(self):
        """回测结束时调用"""
        # 输出回测总结信息
        self.log('回测结束')
        self.log(f'最终资产价值: {self.broker.getvalue():.2f}')


# 假设数据已经加载好，以下是如何使用此策略的示例代码

# 初始化Cerebro引擎
cerebro = bt.Cerebro(stdstats=False)

# 加载数据（假设数据已经准备好）
for stock in daily_price['sec_code'].unique():
    # 日期对齐
    data = pd.DataFrame(index=daily_price.index.unique())
    df = daily_price.query(f"sec_code=='{stock}'")[['open','high','low','close','volume','openinterest']]
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].bfill()
    datafeed = bt.feeds.PandasData(dataname=data_, fromdate=fromdate, todate=todate)
    cerebro.adddata(datafeed, name=stock)

# 设置策略
cerebro.addstrategy(MultiStockStrategy)

# 设置初始资金
cerebro.broker.setcash(100000.0)

# 设置手续费
cerebro.broker.setcommission(commission=0.001)  # 0.1%

# 添加分析器
# cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
# cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
# cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')

# 执行回测
results = cerebro.run(maxcpu=16,runonce=True, preload=True)

# # 输出分析结果
# strat = results[0]
# print(f'夏普比率: {strat.analyzers.sharpe.get_analysis()["sharperatio"]:.3f}')
# print(
#     f'最大回撤: {strat.analyzers.drawdown.get_analysis()["max"]["drawdown"]:.2f}%')
# print(f'年化收益率: {strat.analyzers.returns.get_analysis()["rnorm100"]:.2f}%')

# # 绘制结果图表
# cerebro.plot(style='candlestick', barup='green', bardown='red')

KeyboardInterrupt: 